<h1>Projeto De Previsao de Compras</h1>
<h2>Versao.: 1.3</h2>
<h2>Data.:   15/02/2019  16:20</h2>

<h3>
    TODO:<br/>
    <s>1º Reajustar a média móvel;</s><br/>
    <s>2º Calibrar com o auto-arima os modelos arima e sarima;</s><br/>
    <s>3º Gerar o erro quadrático para cada modelo;</s><br/>
    <s>4º Fazer a seleção das quantidades a serem compradas do modelo que contiver o menor erro;</s><br/>
       5º inserir indices a partir de 01/01/2016;<br/>
       6º estudar casos com quantidades = 0 (interpolação ou 0);<br/>
       7º Rodar o projeto com todos os itens.<br/>
</h3>

In [88]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn("deprecated", DeprecationWarning)
import timeit
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
from math import sqrt, floor, ceil
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima

%matplotlib inline

try:
    dateparse = lambda dates: pd.datetime.strptime(dates, '%d.%m.%Y %H:%M:%S')
    dataFrame = pd.read_csv(r"C:\Macros\VENDA-GAM-343.CSV", sep=";", parse_dates=['dt_venda'], 
                        index_col='dt_venda', date_parser=dateparse)
except:
    dateparse = lambda dates: pd.datetime.strptime(dates, '%d/%m/%Y %H:%M:%S')
    dataFrame = pd.read_csv(r"C:\Macros\VENDA-GAM-343.CSV", sep=";", parse_dates=['dt_venda'], 
                       index_col='dt_venda', date_parser=dateparse)
    


In [2]:
dataFrame[(dataFrame['cd_mercadoria'] == 365573) & (dataFrame['cd_empresa'] == 1)].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 0 entries
Data columns (total 3 columns):
cd_empresa       0 non-null int64
cd_mercadoria    0 non-null int64
qt_vendida       0 non-null int64
dtypes: int64(3)
memory usage: 0.0 bytes


In [89]:
def buscarDataAtual():
    data_atual = datetime.today()
    data_em_texto = '{}-{}-{}'.format(data_atual.year, data_atual.month, '01')
    return data_em_texto

In [90]:
def adicionarIndicesFaltantes(df):
    idx = pd.date_range('2016-01-01', dataAtual, freq='MS')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    return df

In [91]:
def suavizacaoExponencialSimples(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    y_hat_avg = pd.DataFrame(teste.copy())
    model_fit = SimpleExpSmoothing(np.asarray(train)).fit(smoothing_level=0.6,optimized=False)
    y_hat_avg['SES'] = model_fit.forecast(len(teste)) 
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SES',y_hat_avg[i:i+1])
    
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    model_fit = SimpleExpSmoothing(np.asarray(train)).fit(smoothing_level=0.6,optimized=False)
    y_hat_avg['SES'] = model_fit.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SES',y_hat_avg[i:i+1])
    
    return dataFrameMercadoria

In [92]:
def holtLinear(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = Holt(np.asarray(train)).fit(smoothing_level = 0.6,smoothing_slope = 0.1)
    y_hat_avg['Holt_linear'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_linear',y_hat_avg[i:i+1])
        
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = Holt(np.asarray(train)).fit(smoothing_level = 0.6,smoothing_slope = 0.1)
    y_hat_avg['Holt_linear'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_linear',y_hat_avg[i:i+1])
    
    return dataFrameMercadoria

In [93]:
def holtWinter(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ExponentialSmoothing(np.asarray(train) ,seasonal_periods=12,trend='add', seasonal='add' ).fit()
    y_hat_avg['Holt_Winter'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_Winter',y_hat_avg[i:i+1])
        
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ExponentialSmoothing(np.asarray(train) ,seasonal_periods=12,trend='add', seasonal='add' ).fit()
    y_hat_avg['Holt_Winter'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'Holt_Winter',y_hat_avg[i:i+1])

    return dataFrameMercadoria
    

In [105]:
def arima(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    ts = ts.astype('float32')
    
    ts_autoarima = ts[0:len(ts)-1] 
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1] 
    
    try:
        print('inicio try 1')
        modelo_auto = auto_arima(ts_autoarima, m=12, seasonal=True, stationary=True)
        arimaOrder = modelo_auto.order
    
        y_hat_avg = pd.DataFrame(teste.copy())
        fit1 = ARIMA(train, order=arimaOrder).fit(disp=0)
        forecast, stderr, conf = fit1.forecast(len(teste))
        y_hat_avg['ARIMA'] = forecast
        print('fim try 1')
        
    except:
        try:
            print('inicio try 2')
            modelo_auto = auto_arima(ts_autoarima, m=12, seasonal=True, stationary=False, transparams=False)
            arimaOrder = modelo_auto.order
    
            y_hat_avg = pd.DataFrame(teste.copy())
            fit1 = ARIMA(train, order=arimaOrder).fit(disp=0)
            forecast, stderr, conf = fit1.forecast(len(teste))
            y_hat_avg['ARIMA'] = forecast
            print('fim try 2')
        except:
            print('inicio try 3')
            modelo_auto = auto_arima(ts_autoarima, m=12, seasonal=True, stationary=False)
            arimaOrder = modelo_auto.order
    
            y_hat_avg = pd.DataFrame(teste.copy())
            fit1 = ARIMA(train, order=arimaOrder).fit(disp=0)
            forecast, stderr, conf = fit1.forecast(len(teste))
            y_hat_avg['ARIMA'] = forecast
            print('fim try 3')
        
    '''    
    arimaOrder = modelo_auto.order
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ARIMA(train, order=arimaOrder).fit(disp=0)
    forecast, stderr, conf = fit1.forecast(len(teste))
    y_hat_avg['ARIMA'] = forecast
    '''    
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'ARIMA',y_hat_avg[i:i+1])
    
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = ARIMA(train, order=arimaOrder).fit(disp=0)
    forecast, stderr, conf = fit1.forecast(len(teste))
    y_hat_avg['ARIMA'] = forecast
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'ARIMA',y_hat_avg[i:i+1])

    return dataFrameMercadoria
    
    

In [95]:
def sarima(dataFrameMercadoria):
    
    totalLinhas = dataFrameMercadoria.shape[0]
    quantidadeLinhasTreino = floor(totalLinhas-((totalLinhas*20)/100))

    ts = dataFrameMercadoria['qt_vendida']
    ts = ts.astype('float32')
    
    train = ts[0:quantidadeLinhasTreino]
    teste = ts[quantidadeLinhasTreino:len(ts)-1]
    
    modelo_auto = auto_arima(ts, m=12, seasonal=True, trace=False)
    
    sarimaOrder = modelo_auto.order
    seasonalOrder = modelo_auto.seasonal_order
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = sm.tsa.statespace.SARIMAX(train, order=sarimaOrder,seasonal_order=seasonalOrder, 
                                 enforce_stationarity=False, enforce_invertibility=False).fit()
    y_hat_avg['SARIMA'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SARIMA',y_hat_avg[i:i+1])
        
    train = ts[0:len(ts)-1]
    teste = ts[len(ts)-1:len(ts)]
    
    y_hat_avg = pd.DataFrame(teste.copy())
    fit1 = sm.tsa.statespace.SARIMAX(train, order=sarimaOrder,seasonal_order=seasonalOrder, 
                                 enforce_stationarity=False, enforce_invertibility=False).fit()
    y_hat_avg['SARIMA'] = fit1.forecast(len(teste))
    
    for i, hatAvg in y_hat_avg.iterrows():
        dataFrameMercadoria.set_value(y_hat_avg[i:i+1].index.date,'SARIMA',y_hat_avg[i:i+1])
        
    return dataFrameMercadoria
    

In [96]:
def calcularMediaMovel(dataFrameMercadoria):
    j = 1
    while j <= 12:
        
        mediasMoveisCalculada = dataFrameMercadoria['qt_vendida'].rolling(j).mean()
        
        data_minima = mediasMoveisCalculada.index.min()+1
        data_maxima = mediasMoveisCalculada.index.max()
        
        date_index = pd.date_range(data_minima, data_maxima, freq="MS")
        serie = pd.Series(index=date_index)
        
        for i, iValue in serie.iteritems():
            serie[i] = mediasMoveisCalculada[i-1]
          
        dataFrameMercadoria['MM'+str(j)] = serie;

        j += 1
    return dataFrameMercadoria

In [97]:
def calcularErro(dataFrameMercadoria):
    aVendida = df.iloc[-7:-1, 2]
    result = list()
    j = 1
    while j < 17:
        modelo = df.columns[j+2]
        aMM = df.iloc[-7:-1:, j+2].values
        previsao = df.iloc[-1, j+2]
        cd = max(df.cd_empresa.unique())
        produto = max(df.cd_mercadoria.unique())
        erroMM = sqrt(mean_squared_error(aVendida, aMM))
        if len(result) > 0:
            if  result[4] > erroMM:
                result = [cd, produto, modelo, previsao, erroMM]
        elif len(result) == 0:
            result = [cd, produto, modelo, previsao, erroMM]
        j += 1

    return result


In [106]:
#inicio = timeit.default_timer()

#arrayMercadoria = dataFrame['cd_mercadoria'].unique()
#arrayEmpresa = [1, 4, 5]
#arrayMercadoria = [117661, 117679, 117726, 118120, 120795, 126652]
arrayMercadoria = [135334]
arrayEmpresa = [1]
resultDataFrame = list()
arrayDeDataFrame = []

dataAtual = buscarDataAtual()
dataFrame = dataFrame[dataFrame.index != dataAtual]
for emp in arrayEmpresa:
    i = 0
    for i, mer in enumerate(arrayMercadoria):
        df = dataFrame[(dataFrame['cd_mercadoria'] == mer) & 
                      (dataFrame['cd_empresa'] == emp)].sort_values(by=['dt_venda'], ascending=True)
        print(mer)
        if len(df) > 7:
            df = adicionarIndicesFaltantes(df)
            df = calcularMediaMovel(df)
            df = suavizacaoExponencialSimples(df)
            df = holtLinear(df)
            df = holtWinter(df)
            df = arima(df)
            df = sarima(df)
            result = calcularErro(df)
            resultDataFrame.append(result)
dfResult = pd.DataFrame(resultDataFrame)
dfResult.columns = ['cd_empresa', 'cd_mercadoria', 'modelo', 'previsao', 'erroMM']
               
#dfResult
#fim = timeit.default_timer()
#print(f'Tempo de execução: {(fim - inicio)/60} minutos')

135334
inicio try 1
inicio try 2
inicio try 3


ValueError: The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.

In [103]:
dfResult

,cd_empresa,cd_mercadoria,modelo,previsao,erroMM
0,1,143866,ARIMA,285.598866,176.411127
